In [1]:
#imports 
import requests 
import re
import random
import time
import faker
import numpy as np
import pandas as pd
import array as arr
import selenium 
from selenium import webdriver
from selenium.webdriver.support.select import Select
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from faker import Faker
from bs4 import BeautifulSoup

In [2]:
def brokerPage(brokerCompany):
    #to store scrapped data
    compName = None 
    address = None 
    activeListings = None 
    PhoneNumber= None
    About = None 

    df2 = pd.DataFrame()

    response = requests.get(brokerCompany, headers=HEADERS)
    soup = BeautifulSoup(response.content)
    bInfo = soup.find(class_ = "container container--full-width container--grid")
    if bInfo: 
        bInfoRows = list(bInfo.children)
        bInfoRowsNum = len(bInfoRows)
        span = []
    
        aboutTag = soup.find(class_ = "bio-info__description")
        if aboutTag:
            About = (re.sub(r"[\r\n\t]*", "", aboutTag.text))
            
        for i in range(bInfoRowsNum):
            bInfoRow = bInfoRows[i]
            
            for data in bInfoRow.children:
        
                compNameTag = data.find("h1")
                if compNameTag:
                    compName = compNameTag.text
        
                spanTag = data.find_all("span")
                
                for i in spanTag:
                    span.append(i.text)
                    
        activeListings = span[1]
        address = (re.sub(r"[\r\n\t]*", "", span[3]))
        phoneNumber = span[5]
                
        dataframe2 = [{"company name" : compName, "address" : address, "phone number" : phoneNumber, "active listings" : activeListings, 'About' : About}]
        df2 = df2.append(dataframe2, ignore_index=True)
    return df2

In [40]:
df1 = pd.DataFrame() #for property
df4 = pd.DataFrame() #for agent
df5 = pd.DataFrame() #for project
df6 = pd.DataFrame() #for controlled by
df9 = pd.DataFrame() #for amenities
x = pd.DataFrame() #for broker company
df3 = pd.DataFrame() #for user
df7 = pd.DataFrame() #for reviews
df8 = pd.DataFrame() #for veiwing
df10 = pd.DataFrame() #for main areas of focus
#to store scrapped data
pArea = None
location = None
price = None
propertyType = None
bedrooms = None
bathrooms = None
listingDate = None
propertySize = None
rentOrBuy = None
companyName = None
agentName = None
projectName = None
amenities = []
propertyDescription = None 
propertyName = None
paymentMethod = None 
downPayment = None
brokerCompany = None
contactNumber = None
contactNumberList = []
propertyNumberList = []
areasList = []

In [174]:
allPropertiesLinks = []
HEADERS = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36 Edg/106.0.1370.42'}
mainURL = 'https://www.propertyfinder.eg/'
for pages in range (1,246):
    pageHtml = requests.get(f'https://www.propertyfinder.eg/en/search?c=1&ob=nd&page={pages}&pf=100000&pt=3000000', headers = HEADERS)
    soup = BeautifulSoup(pageHtml.content, 'lxml')
    allProperties = soup.find_all('div', class_ = 'card-list__item')
    for oneProperty in allProperties:
        links = oneProperty.find('a', href = True)
        for link in oneProperty.find_all('a', href = True):
            allPropertiesLinks.append(mainURL + link['href'])
print(len(allPropertiesLinks)) 
pageNumber = 0

6125


In [130]:
for page in allPropertiesLinks:
    print(pageNumber)
    propertyNumberList.append(pageNumber)
    driver = webdriver.Chrome("/Users/mernaabdelbadie/Downloads/chromedriver")
    driver.get(page)
    response = requests.get(page, headers = HEADERS)
    soup = BeautifulSoup(response.content)
    info = soup.find(class_ = "property-page__column--left")
    
    if info:
        infoRows = list(info.children)
        infoRowsNum = len(infoRows)
        projectName = None
        #to get downpayment and payment method
        try:
            downPaymentTag = driver.find_element(By.CLASS_NAME, 'property-payment-plan__value')
            downPayment = downPaymentTag.text
            paymentMethod = "Installments"
        except NoSuchElementException:
            downPayment = None 
            paymentMethod = "cash"
        
        #to get contact number
        if not(soup.find(class_ = "property-gone__area")):
            driver.execute_script("document.getElementsByClassName('button-2 button-primary property-contact__primary-button')[0].click();")
            contactResponse = driver.page_source
            contactSoup = BeautifulSoup(contactResponse, 'html.parser')
            if contactSoup.find(class_="text text--size4 text--bold property-contact__number"):
                contactNumber = contactSoup.find(class_="text text--size4 text--bold property-contact__number").contents[0]
                driver.quit()
            else:
                driver.quit()
        

        rentOrBuyTag = soup.find(class_= "menu__link menu__link--active")
        if rentOrBuyTag:
            rentOrBuy = rentOrBuyTag.text 

            
        descriptionTag = soup.find(class_ = "property-page__description")
        if descriptionTag:
            propertyDescription = (re.sub(r"[\n\t]*", "", descriptionTag.text))
            
        propertyNameTag = soup.find("h2")
        if propertyNameTag:
            propertyName = (re.sub(r"[\n\t]*", "", propertyNameTag.text))

        for i in range(infoRowsNum):
            infoRow = infoRows[i]
            for data in infoRow.children:
            
                pAreaTag = data.find (class_ = "text text--size3 property-location__tower-name")
                if pAreaTag:
                    pArea = pAreaTag.text
                    areasList.append(pArea)
            
                locationTag = data.find(class_ = "property-location__detail-area")
                if locationTag:
                    location = locationTag.text
            
                priceTag = data.find(class_ = "property-price__price")
                if priceTag:
                    price = (re.sub(r"[\n\t\s]*", "", priceTag.text))
        
                facts = []
                factsTag = data.find_all(class_ = "property-facts__value") 
                for i in factsTag:
                    if(len(factsTag) > 0):
                        propertyType = (re.sub(r"[\n\t]*", "", factsTag[0].text))
                    if(len(factsTag) > 1):
                        propertySize = (re.sub(r"[\n\t]*", "", factsTag[1].text))
                    if(len(factsTag) > 2):
                        bedrooms = (re.sub(r"[\n\t]*", "", factsTag[2].text))
                    if (len(factsTag) > 3):
                        bathrooms = (re.sub(r"[\n\t]*", "", factsTag[3].text))
                    if (len(factsTag) > 4):
                        projectName = (re.sub(r"[\n\t]*", "", factsTag[4].text))


                legalContentTag = data.find_all('li', class_ = "property-page__legal-list-item")
    
                for i in legalContentTag:
                    listingDate = legalContentTag[1].text
                    listingDate = listingDate[7:]
        
                amenitiesTag = data.find_all(class_ = "property-amenities__list")
                if amenitiesTag:
                    for i in amenitiesTag:
                        amenities.append(re.sub(r"[\n\t]*", "", i.text))
                
                companyNameTag = data.find(class_ = "property-agent__position-broker-name")
                if companyNameTag:
                    companyName = (re.sub(r"[\n\t]*", "", companyNameTag.text))
        
                agentNameTag = data.find("h4")
                if agentNameTag:
                    agentName = agentNameTag.text
                    
                #to get broker company
                brokerCompanyTag = data.find_all('a', href=True)
                if brokerCompanyTag:
                    brokerCompany = "https://www.propertyfinder.eg/" + brokerCompanyTag[0]['href']
    
        x = x.append(brokerPage(brokerCompany), ignore_index=True)
        
 
        dataframe1 = [{"property number" : pageNumber, "property name" : propertyName, "rent or buy": rentOrBuy, "listing date": listingDate, "area" : pArea, "location" : location, "price" : price, "propertyType" : propertyType, "bedrooms" : bedrooms, "bathrooms" : bathrooms, "size" : propertySize, "down payment" : downPayment, "payment method" : paymentMethod, "contact number" : contactNumber, "project" : projectName, "description" : propertyDescription[11:]}]
        df1 = df1.append(dataframe1, ignore_index=True)
            
        dataframe4 = [{"contact number" : contactNumber, "agent name" : agentName, "companyName" : companyName}]
        df4 = df4.append(dataframe4, ignore_index=True)
        for a in amenities:
            dataframe9 = [{"property number" : pageNumber, "amenity" : a}]
            df9 = df9.append(dataframe9, ignore_index=True)
        if projectName: 
                dataframe5 = [{"project name" : projectName, "location" : location}]
                df5 = df5.append(dataframe5, ignore_index=True)
                dataframe6 = [{"project name" : projectName, "company name" : companyName}]
                df6 = df6.append(dataframe6, ignore_index=True)
        amenities = []
        pageNumber = pageNumber + 1

5914


<ipython-input-130-d366bbbaeaf7>:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("/Users/mernaabdelbadie/Downloads/chromedriver")


5915
5916
5917
5918
5919
5920
5921
5922
5923
5924
5925
5926
5927
5928
5929
5930
5931
5932
5933
5934
5935
5936
5937
5938
5939
5940
5941
5942
5943
5944
5945
5946
5947
5948
5949
5950
5951
5952
5953
5954
5955
5956
5957
5958
5959
5960
5961
5962
5963
5964
5965
5966
5967
5968
5969
5970
5971
5972
5973
5974
5975
5976
5977
5978
5979
5980
5981
5982
5983
5984
5985
5986
5987
5988
5989
5990
5991
5992
5993
5994
5995
5996
5997
5998
5999
6000
6001
6002
6003
6004
6005
6006
6007
6008
6009
6010
6011
6012
6013
6014
6015
6016
6017
6018
6019
6020
6021
6022
6023
6024
6025
6026
6027
6028
6029
6030
6031
6032
6033
6034
6035
6036
6037
6038
6039
6040
6041
6042
6043
6044
6045
6046
6047
6048
6049
6050
6051
6052
6053
6054
6055
6056
6057
6058
6059
6060
6061
6062
6063
6064
6065
6066
6067
6068
6069
6070
6071
6072
6073
6074
6075
6076
6077
6078
6079
6080
6081
6082
6083
6084
6085
6086
6087
6088
6089
6090
6091
6092
6093
6094
6095
6096
6097
6098
6099
6100
6101
6102
6103
6104
6105
6106
6107
6108
6109
6110
6111
6112
6113
6114


In [175]:
#to store user information
email = None
gender = None
age = None
main_areas_of_focus = None
username = None
textualReviews = None
rating = None
contactNo = None
emailList = []

df3 = pd.DataFrame()
df10 = pd.DataFrame()
df7 = pd.DataFrame()
df8 = pd.DataFrame()

faker = Faker()
for i in range(0,100):
    contactNo = random.choice(contactNumberList)
    gender = random.choice(['M', 'F'])
    if (gender == 'M'):
        username = faker.name_male()
    else:
        username = faker.name_female()
    email = faker.email()
    emailList.append(email)
    age = random.randrange(20, 50, 3)
    textualReviews = random.choice(['good', 'amazing', 'not good', 'did not like it','overpriced', 'very nice','helpful agent', 'beautiful view'])
    rating = random.choice(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10'])
    main_areas_of_focus = random.choice(areasList)
    dataframe3 = [{"username" : username, "email" : email, "age" : age, "gender" : gender}]
    dataframe7 = [{"textual reviews" : textualReviews, "rating" : rating, "email" : email, "contact number" : contactNo}]
    dataframe8 = [{"property number" : random.choice(propertyNumberList), "email" : email}]
    dataframe10 = [{"email" : random.choice(emailList), "area" : main_areas_of_focus}]
    df10 = df10.append(dataframe10, ignore_index=True)
    df8 = df8.append(dataframe8, ignore_index=True)
    df3 = df3.append(dataframe3, ignore_index=True)
    df7 = df7.append(dataframe7, ignore_index=True)

In [184]:
df1

,property number,property name,rent or buy,listing date,area,location,price,propertyType,bedrooms,bathrooms,size,down payment,payment method,contact number,project,description
0,0,Apartment for Sale in District 3...,Buy,6 hours ago,District 300,"District 300Giza, 6 October City, Northern Exp...",Askforprice,Apartment,3,2,"1,722sqft / 160sqm",NaN,cash,+201113636207,NaN,استلم فورا شقه 160م بمقدم 35% وقسط يصل الي 3 ...
1,1,Penthouse for Sale in The Square...,Buy,6 hours ago,The Square,"The SquareCairo, New Cairo City, The 5th Settl...",Askforprice,Penthouse,3,3,"1,830sqft / 170sqm",NaN,cash,+201022225192,NaN,The SquarePenthouse for sale Very prime locati...
3,3,Apartment for Sale in Fifth Squa...,Buy,6 hours ago,Fifth Square,"Fifth SquareCairo, New Cairo City, North Inves...","405,900EGP",Apartment,3,3,"2,637sqft / 245sqm",NaN,cash,+201234555575,NaN,Details Apartment For sale In El Marassem comp...
4,4,Townhouse for Sale in Sodic East...,Buy,6 hours ago,Sodic East,"Sodic EastCairo, New Heliopolis, 6th District","260,000EGP",Townhouse,3+Maid,3,"2,207sqft / 205sqm",NaN,cash,+201050005001,NaN,Fully finished sky house in Sodic East with ro...
5,5,"Apartment for Sale in Sarai, Mos...",Buy,6 hours ago,Sarai,"SaraiCairo, Mostakbal City - Future City, Most...","1,537,000EGP",Apartment,3,2,"1,593sqft / 148sqm",NaN,cash,+201092166775,NaN,كمبوند سراي القاهرة الجديدة بجوار مدينتي تقاطع...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6132,6134,Apartment for Sale in Al Imam Ma...,Buy,12 hours ago,Al Imam Malik St.,"Al Imam Malik St.Qalyubia, Obour City, 6th Dis...","1,650,000EGP",Apartment,3,2,"2,368sqft / 220sqm",None,cash,+201050404653,None,شقه للبيع بالعبور الحي السادسقريبه من الرئيسي ...
6133,6135,Apartment for Sale in Alexandria...,Buy,12 hours ago,Alexandria St.,"Alexandria St.Matrouh, Marsa Matrouh","100,000EGP",Apartment,3,2,"1,453sqft / 135sqm",None,cash,+20221257363,None,شقتــــــــــــك عــــنــدنــــــا سكـــــــــ...
6134,6136,"Villa for Sale in Joulz, Cairo A...",Buy,11 hours ago,Joulz,"JoulzGiza, 6 October City, Cairo Alexandria De...","1,350,000EGP",Villa,5+Maid,5,"5,328sqft / 495sqm",None,cash,+201222204294,None,1 YEAR DELIVERY & PRIME LOCATION VILLA▪️ Locat...
6135,6137,"Apartment for Sale in Sarai, Mos...",Buy,12 hours ago,Sarai,"SaraiCairo, Mostakbal City - Future City, Most...","1,550,000EGP",Apartment,2,2,"1,324sqft / 123sqm",None,cash,+20221257104,None,"Own your apartment in Sarai Compound, Fifth Se..."


In [185]:
df7  = df7.drop_duplicates()
df7

,textual reviews,rating,email,contact number
0,amazing,6,crystal15@example.net,201113148321
1,beautiful view,10,jason41@example.net,201024246479
2,good,10,mcdonaldmichael@example.net,201110777016
3,overpriced,10,onealteresa@example.com,201008823478
4,helpful agent,7,clarsen@example.org,201234555575
...,...,...,...,...
95,overpriced,3,brianromero@example.com,201118321817
96,helpful agent,7,rickythomas@example.com,20221257468
97,amazing,5,james35@example.net,201145541993
98,not good,3,brewerjordan@example.org,201063712494


In [186]:
df6 = df6.drop_duplicates()
df6

,project name,company name
0,La Verde Developments,La Verde Developments
1,Al-Masryia Group,Al-Masryia Group
2,Plaza Gardens,Plaza Gardens


In [188]:
df5 = df5.drop_duplicates()
df5

,project name,location
0,La Verde Developments,"La VerdeCairo, New Capital City, New Capital C..."
1,Al-Masryia Group,"IsolaGiza, 6 October City, Hadayek October"
3,Plaza Gardens,"RhodesCairo, New Capital City, New Capital Com..."


In [189]:
df4 = df4.drop_duplicates()
df4

,contact number,agent name,companyName
0,+201113636207,Elmostafa Realstate,Elmostafa Realstate
1,+201022225192,Salah Team,Green Homes Real estate Agency
2,+201100252748,Hassan sobhy,Property hills Plus
3,+201234555575,Nermine Khaled,Evernest Estate
4,+201050005001,Sara Samir,Hills Estate
...,...,...,...
3370,+201004375555,Mai Adel,Shahba properties
3381,+201003255094,Ibrahim Abo Hashima,I Zone
3386,+201155899448,alaa ahmed,EDGE Investment
3387,+201002503069,mohamed mosaad,Abrag Real Estate


In [190]:
df8  = df8.drop_duplicates()
df8

,property number,email
0,1083,crystal15@example.net
1,395,jason41@example.net
2,944,mcdonaldmichael@example.net
3,24,onealteresa@example.com
4,540,clarsen@example.org
...,...,...
95,181,brianromero@example.com
96,433,rickythomas@example.com
97,698,james35@example.net
98,763,brewerjordan@example.org


In [191]:
df9 = df9.drop_duplicates()
df9

,property number,amenity
0,1,Unfurnished
1,1,Security
2,1,Children's Pool
3,1,Concierge
4,1,Maid Service
...,...,...
64277,6136,Pets Allowed
64278,6136,View of Water
64279,6136,View of Landmark
64280,6136,Covered Parking


In [192]:
df10 = df10.drop_duplicates()
df10

,email,area
0,crystal15@example.net,Telal Al Sokhna
1,crystal15@example.net,El Rehab Extension
2,crystal15@example.net,Park Side Residence
3,crystal15@example.net,Seazen
4,onealteresa@example.com,Mountain View iCity
...,...,...
93,nicholas58@example.com,Porto New Cairo
94,angel94@example.net,Nasr Ahmed Zaki St.
95,sarah70@example.com,R7
96,omitchell@example.net,Lake View Residence


In [169]:
df4.to_csv("Agent Data.csv", sep = ',', encoding = 'utf-8', index = False)
df5.to_csv("Project Data.csv", sep = ',', encoding = 'utf-8', index = False)
df6.to_csv("Controlled By Data.csv", sep = ',', encoding = 'utf-8', index = False)
df7.to_csv("Reviews Data.csv", sep = ',', encoding = 'utf-8', index = False)
df8.to_csv("Veiwing Data.csv", sep = ',', encoding = 'utf-8', index = False)
x.to_csv("Broker Company Data.csv", sep = ',', encoding = 'utf-8', index = False)
df1.to_csv("Property Data.csv", sep = ',', encoding = 'utf-8', index = False)
df3.to_csv("User Data.csv", sep = ',', encoding = 'utf-8', index = False)
df9.to_csv("Amenities.csv", sep = ',', encoding = 'utf-8', index = False)
df10.to_csv("Main Areas of Focus.csv", sep = ',', encoding = 'utf-8', index = False)

In [193]:
df3

,username,email,age,gender
0,Sheena Fischer,crystal15@example.net,32,F
1,Julie Melendez,jason41@example.net,44,F
2,Lisa Horne,mcdonaldmichael@example.net,23,F
3,Amy Moore,onealteresa@example.com,35,F
4,Kyle Anderson,clarsen@example.org,35,M
...,...,...,...,...
95,Kristin Stone,brianromero@example.com,35,F
96,Randy Donovan,rickythomas@example.com,20,M
97,Ryan Smith,james35@example.net,35,M
98,Daniel Williamson,brewerjordan@example.org,20,M


In [194]:
x = x.drop_duplicates()
x

,company name,address,phone number,active listings,About
0,Elmostafa Realstate,العنوان / 6اكتوبر الحي (11) ابراج سيتي ستار بر...,+201050382245,325 Properties,NaN
1,Green Homes Real estate Agency,"Skies Mall, South Tessen, New Cairo, Egypt",+201096963555,963 Properties,We Green Homes who are attempting to enter the...
2,Property hills Plus,"30, , Sheikh Zayed City, 7th District, Giza, E...",+201155468540,44 Properties,NaN
3,Evernest Estate,Waterway,+201234555575,151 Properties,NaN
4,Hills Estate,٣٠٥ شارع التسعين الجنوبي مبنى جرين تاور، الدور...,+201050005001,72 Properties,NaN
...,...,...,...,...,...
1812,Guides Real Estate,٤١ شارع جزيرة العرب,+201226080060,185 Properties,GUIDES REAL ESTATE (GRE)Learn more about our c...
1815,Vertex Real Estate,New Cairo,+201116511600,150 Properties,
1837,Mas Egypt Real Estate,"20 Elfalah St., from Shehab st. Mohandseen.Egy...",+201030001384,265 Properties,Mas Egypt Real Estate is an established real e...
1846,Real Estate Corner,"elnarges Emarat 1,1 new cairo",+20106644221,293 Properties,Is your ultimate destination Offering Property...
